In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3

In [17]:
url = "https://www.elections.tn.gov.in/ACwise_Gendercount_19122025.aspx"

response = requests.get(url, timeout=30)
response.raise_for_status()

html = response.text

In [18]:
soup = BeautifulSoup(html, "html.parser")

table = soup.find("table")

if table is None:
    raise ValueError("No table found on the page")


In [19]:
rows = table.find_all("tr")
rows


[<tr>
 <th style="text-align:center">District No.</th>
 <th style="text-align:center">District Name</th>
 <th style="text-align:center">AC No.</th>
 <th style="text-align:center">Name of Assembly Constituency</th>
 <th style="text-align:center">Male</th>
 <th style="text-align:center">Female</th>
 <th style="text-align:center">Third Gender</th>
 <th style="text-align:center">Total</th>
 </tr>,
 <tr>
 <td rowspan="10" style="vertical-align:middle; text-align:center">1</td>
 <td rowspan="10" style="vertical-align:middle; font-weight:bold">Thiruvallur</td>
 <td style="vertical-align:middle; text-align:center">1</td>
 <td>Gummidipoondi</td>
 <td style="text-align:right">118205</td>
 <td style="text-align:right">124318</td>
 <td style="text-align:right">35</td>
 <td style="text-align:right">242558</td>
 </tr>,
 <tr>
 <td style="vertical-align:middle; text-align:center">2</td>
 <td>Ponneri</td>
 <td style="text-align:right">116890</td>
 <td style="text-align:right">122218</td>
 <td style="te

In [20]:
data = []

for row in rows[1:]:
    cols = row.find_all("td")

    if len(cols) >= 6:
        ac_no   = cols[-6].get_text(strip=True)
        ac_name = cols[-5].get_text(strip=True)
        total   = cols[-1].get_text(strip=True)

        data.append([ac_no, ac_name, total])


In [21]:
df_sir_2026 = pd.DataFrame(
    data,
    columns=[
        "ac_no",
        "ac_name",
        "total_eligible_voters_2026"
    ]
)

df_sir_2026.head(10)


,ac_no,ac_name,total_eligible_voters_2026
0,1,Gummidipoondi,242558
1,2,Ponneri,239141
2,3,Tiruttani,252154
3,4,Thiruvallur,238818
4,5,Poonamallee,348034
5,6,Avadi,402693
6,7,Maduravoyal,325407
7,8,Ambattur,304511
8,9,Madavaram,386327
9,10,Thiruvottiyur,222806


In [22]:
# AC number
df_sir_2026["ac_no"] = pd.to_numeric(
    df_sir_2026["ac_no"],
    errors="coerce"
)

# remove commas from voter count
df_sir_2026["total_eligible_voters_2026"] = (
    df_sir_2026["total_eligible_voters_2026"]
    .str.replace(",", "", regex=False)
)

df_sir_2026["total_eligible_voters_2026"] = pd.to_numeric(
    df_sir_2026["total_eligible_voters_2026"],
    errors="coerce"
)

df_sir_2026 = df_sir_2026.dropna(
    subset=["ac_no", "total_eligible_voters_2026"]
)

df_sir_2026["ac_no"] = df_sir_2026["ac_no"].astype(int)
df_sir_2026["total_eligible_voters_2026"] = (
    df_sir_2026["total_eligible_voters_2026"].astype(int)
)

df_sir_2026.head()


,ac_no,ac_name,total_eligible_voters_2026
0,1,Gummidipoondi,242558
1,2,Ponneri,239141
2,3,Tiruttani,252154
3,4,Thiruvallur,238818
4,5,Poonamallee,348034


In [23]:
df_sir_2026.shape


(234, 3)

In [24]:
conn = sqlite3.connect("../db/tn_election.db")
print("Connected to SQLite")


Connected to SQLite


In [25]:
df_sir_2026.to_sql(
    "voters_2026",
    conn,
    if_exists="replace",
    index=False
)

print("voters_2026 table created/updated")


voters_2026 table created/updated


In [26]:
pd.read_sql("SELECT * FROM voters_2026",conn)


,ac_no,ac_name,total_eligible_voters_2026
0,1,Gummidipoondi,242558
1,2,Ponneri,239141
2,3,Tiruttani,252154
3,4,Thiruvallur,238818
4,5,Poonamallee,348034
...,...,...,...
229,41,Ranipet,235960
230,42,Arcot,235816
231,160,Sirkazhi,232766
232,161,Mayiladuthurai,225840
